## Install Required Dependencies

Install the necessary packages for quantization-aware training including ipywidgets for notebook UI, nvidia-modelopt for quantization tools, and trl for transformer training.

In [1]:
%pip install ipywidgets nvidia-modelopt[all] trl

Note: you may need to restart the kernel to use updated packages.


## Load Environment Variables and Configure Model

Load Hugging Face authentication token from environment and specify the base model to use for quantization-aware training.

In [2]:
from dotenv import load_dotenv
load_dotenv()
import os

os.environ["HF_TOKEN"] = os.getenv("HF_WRITE_TOKEN")

model_name = "meta-llama/Llama-3.1-8B-Instruct"

## Enable Hugging Face Checkpointing

Enable ModelOpt's Hugging Face checkpointing feature to ensure quantized models can be properly saved and loaded with Hugging Face's transformers library.

In [3]:
import modelopt.torch.opt as mto
mto.enable_huggingface_checkpointing()

/root/src/github.com/elizabetht/language-modeling-from-scratch/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GB10 which is of cuda capability 12.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (8.0) - (12.0)
    
  warnings.warn(
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


Skipping import of cpp extensions due to incompatible torch version 2.9.0+cu130 for torchao version 0.15.0             Please see https://github.com/pytorch/ao/issues/2919 for more info


ModelOpt save/restore enabled for `transformers` library.


ModelOpt save/restore enabled for `diffusers` library.


ModelOpt save/restore enabled for `peft` library.


## Configure Model Arguments

Define model configuration including attention implementation, precision (bfloat16), and device settings. Setting `use_cache=False` is required for training to enable gradient computation.

In [4]:
import torch
from trl import ModelConfig

model_args = ModelConfig(
    model_name_or_path=model_name,
    attn_implementation="eager",
    torch_dtype=torch.bfloat16,
)
model_kwargs = {
    "revision": model_args.model_revision,
    "trust_remote_code": model_args.trust_remote_code,
    "attn_implementation": model_args.attn_implementation,
    "torch_dtype": model_args.torch_dtype,
    "use_cache": False,
    "device_map": "auto",
}

## Load Model and Tokenizer

Load the pre-trained Llama model and its corresponding tokenizer from Hugging Face Hub using the configurations defined above.

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(model_args.model_name_or_path, **model_kwargs)

tokenizer = AutoTokenizer.from_pretrained(
    model_args.model_name_or_path,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Configure Dataset Arguments

Specify the dataset to use for training (Multilingual-Thinking dataset) and define the train/test split configuration with 10% held out for evaluation.

In [6]:
from trl import ScriptArguments

script_args = ScriptArguments(
    dataset_name="HuggingFaceH4/Multilingual-Thinking",
    dataset_train_split="train",
    dataset_test_split="test",
)
test_size = 0.1


## Load and Split Dataset

Load the multilingual reasoning dataset and split it into training and evaluation sets using a fixed seed for reproducibility.

In [7]:
from datasets import load_dataset

dataset = load_dataset(script_args.dataset_name)
# split the dataset into train and test
dataset = dataset[script_args.dataset_train_split].train_test_split(test_size=test_size, seed=42)
train_dataset = dataset[script_args.dataset_train_split]
eval_dataset = dataset[script_args.dataset_test_split]

## Configure Training Arguments

Set up training hyperparameters including learning rate, batch size, evaluation strategy, and checkpointing configuration. The training will run for 1 epoch with evaluation every 50 steps.

In [8]:
from trl import SFTConfig

training_args = SFTConfig(
    output_dir="output",
    num_train_epochs=5,
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    max_length=4096,
    warmup_ratio=0.1,
    eval_strategy="steps",
    eval_on_start=True,
    logging_steps=25,
    save_steps=450,
    eval_steps=50,
    save_total_limit=2,
)

## Initialize SFT Trainer

Create a Supervised Fine-Tuning (SFT) trainer that will handle the training loop, evaluation, and model optimization during quantization-aware training.

In [9]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset[script_args.dataset_train_split],
    eval_dataset=dataset[script_args.dataset_test_split],
    processing_class=tokenizer,
)

## Setup Quantization Configuration

Configure quantization settings (NVFP4 format) and prepare calibration data. The forward loop function is defined to run calibration samples through the model for quantization parameter estimation.

In [10]:
import torch

import modelopt.torch.quantization as mtq

# Some configs don't need calibration, but other quantization configurations may require it.
quantization_config = mtq.FP8_DEFAULT_CFG
calib_size = 512

dataset = torch.utils.data.Subset(
    trainer.eval_dataset, list(range(min(len(trainer.eval_dataset), calib_size)))
)
data_loader = trainer.get_eval_dataloader(dataset)


def forward_loop(model):
    for data in data_loader:
        model(**data)

## Apply Quantization to Model

Apply the quantization configuration to the model using the calibration data. This prepares the model for quantization-aware training by inserting quantization layers and running calibration.

In [11]:
mtq.quantize(model, quantization_config, forward_loop)

Registered <class 'transformers.models.llama.modeling_llama.LlamaAttention'> to _QuantAttention for KV Cache quantization
Inserted 771 quantizers


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): QuantLlamaAttention(
          (q_proj): QuantLinear(
            in_features=4096, out_features=4096, bias=False
            (input_quantizer): TensorQuantizer((4, 3) bit fake per-tensor amax=8.3750 calibrator=MaxCalibrator quant)
            (output_quantizer): TensorQuantizer(disabled)
            (weight_quantizer): TensorQuantizer((4, 3) bit fake per-tensor amax=0.7461 calibrator=MaxCalibrator quant)
          )
          (k_proj): QuantLinear(
            in_features=4096, out_features=1024, bias=False
            (input_quantizer): TensorQuantizer((4, 3) bit fake per-tensor amax=8.3750 calibrator=MaxCalibrator quant)
            (output_quantizer): TensorQuantizer(disabled)
            (weight_quantizer): TensorQuantizer((4, 3) bit fake per-tensor amax=0.6992 calibrator=MaxCalibrator quant)
          )
          (v_proj

## Start Quantization-Aware Training

Configure CUDA environment variables for proper compilation and start the training process. The model will learn while simulating the effects of quantization.

In [12]:
import os
os.environ["TORCH_CUDA_ARCH_LIST"] = "12.1"
os.environ["TRITON_PTXAS_PATH"] = "/usr/local/cuda/bin/ptxas"
os.environ["PATH"] = "/usr/local/cuda/bin:" + os.environ.get("PATH", "")
os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda/lib64:" + os.environ.get("LD_LIBRARY_PATH", "")

trainer.train()

Loading extension modelopt_cuda_ext_fp8...


Loaded extension modelopt_cuda_ext_fp8 in 0.0 seconds


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
0,No log,2.256705,1.331495,0.000000,0.574359
50,1.120400,1.029272,1.007731,51429.000000,0.717016
100,0.949200,0.999120,1.062640,110540.000000,0.720615
150,1.123900,1.075860,1.037810,165574.000000,0.707689
200,1.256000,1.233319,1.116227,220373.000000,0.684657
250,1.352100,1.339444,1.291183,272024.000000,0.666098
300,1.514800,1.350784,1.153571,328039.000000,0.668970
350,1.384000,1.373163,1.329398,386991.000000,0.661902
400,1.407600,1.355927,1.248261,441345.000000,0.666697
450,1.544000,1.319960,1.399849,499140.000000,0.669181


Saved ModelOpt state to output/checkpoint-450/modelopt_state.pth


Saved ModelOpt state to output/checkpoint-900/modelopt_state.pth


Saved ModelOpt state to output/checkpoint-1350/modelopt_state.pth


Saved ModelOpt state to output/checkpoint-1800/modelopt_state.pth


Saved ModelOpt state to output/checkpoint-2250/modelopt_state.pth


TrainOutput(global_step=2250, training_loss=0.5248833482530382, metrics={'train_runtime': 12023.6405, 'train_samples_per_second': 0.374, 'train_steps_per_second': 0.187, 'total_flos': 1.123802430234624e+17, 'train_loss': 0.5248833482530382, 'epoch': 5.0})

In [13]:
import torch
from modelopt.torch.export import export_hf_checkpoint

export_path = "./output/fp8"
export_hf_checkpoint(model, dtype=torch.bfloat16, export_dir=export_path)
tokenizer.save_pretrained(export_path)

/root/src/github.com/elizabetht/language-modeling-from-scratch/.venv/lib/python3.12/site-packages/modelopt/torch/export/unified_export_hf.py:413: UserWarning: Model's original dtype (bfloat16) differs from target dtype (torch.bfloat16), which may lead to numerical errors.
  warnings.warn(
/root/src/github.com/elizabetht/language-modeling-from-scratch/.venv/lib/python3.12/site-packages/modelopt/torch/export/unified_export_hf.py:269: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  weight_scaling_factor = torch.tensor(weight_quantizer.amax / weight_quantizer.maxbound)


('./output/fp8/tokenizer_config.json',
 './output/fp8/special_tokens_map.json',
 './output/fp8/chat_template.jinja',
 './output/fp8/tokenizer.json')

In [14]:
import os

# Save model config
model.config.save_pretrained(export_path)
print("✓ Saved config")

# Create a README model card
model_card = """---
license: llama3.1
base_model: meta-llama/Llama-3.1-8B-Instruct
tags:
  - llama
  - quantized
  - nvidia-modeloptimizer
  - FP8
  - QAT 
library_name: nvidia-modeloptimizer
---

# Llama-3.1-8B-Instruct Quantized (ModelOpt FP8) through QAT

This is a quantized version of [meta-llama/Llama-3.1-8B-Instruct](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct) using [modelopt](https://github.com/NVIDIA/Model-Optimizer) with NVFP4 weight quantization.

## Model Details

- **Base Model:** meta-llama/Llama-3.1-8B-Instruct
- **Quantization Method:** modelopt FP8 Quantization Aware Training (QAT) 
- **Weight Precision:** FP8
- **Original Size:** ~16 GB (bfloat16)
- **Quantized Size:** ~6 GB (fp8)

## Usage

```python
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load base model structure
model = AutoModelForCausalLM.from_pretrained(
    "tokenlabsdotrun/Llama-3.1-8B-ModelOpt-FP8",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True
)

# Load tokenizer and generate
tokenizer = AutoTokenizer.from_pretrained("tokenlabsdotrun/Llama-3.1-8B-ModelOpt-NVFP4-QAT")

inputs = tokenizer("Hello, my name is", return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=10)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
```

## License

This model inherits the [Llama 3.1 Community License](https://llama.meta.com/llama3_1/license/).
"""

with open(f"{export_path}/README.md", "w") as f:
    f.write(model_card)
print("✓ Created model card")

print(f"\nModel saved to {export_path}/")
print("Contents:", os.listdir(export_path))

✓ Saved config
✓ Created model card

Model saved to ./output/fp8/
Contents: ['special_tokens_map.json', 'README.md', 'model-00002-of-00002.safetensors', 'tokenizer_config.json', 'chat_template.jinja', 'hf_quant_config.json', 'generation_config.json', 'config.json', 'model-00001-of-00002.safetensors', 'model.safetensors.index.json', 'tokenizer.json']


In [15]:
from huggingface_hub import create_repo, upload_folder

# Load write token from .env file
hf_write_token = os.getenv("HF_WRITE_TOKEN")

# Upload to HuggingFace Hub
repo_name = "tokenlabsdotrun/Llama-3.1-8B-ModelOpt-FP8-QAT"  # Change to your username/repo

try:
    # Create the repo (set private=True if you want it private)
    create_repo(repo_name, exist_ok=True, private=False, token=hf_write_token)
    print(f"✓ Repository created: {repo_name}")
    
    # Upload all files
    upload_folder(
        folder_path=export_path,
        repo_id=repo_name,
        repo_type="model",
        commit_message="Upload Llama-3.1-8B quantized with ModelOpt FP8-QAT",
        token=hf_write_token
    )
    print(f"✓ Uploaded to https://huggingface.co/{repo_name}")
    
except Exception as e:
    print(f"❌ Error: {e}")

✓ Repository created: tokenlabsdotrun/Llama-3.1-8B-ModelOpt-FP8-QAT


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

✓ Uploaded to https://huggingface.co/tokenlabsdotrun/Llama-3.1-8B-ModelOpt-FP8-QAT


In [16]:
import sys
sys.path.append("../..")  # Add parent directory to path

# Force reload the module to pick up changes
import importlib
import quantization_theory_helper
importlib.reload(quantization_theory_helper)

from quantization_theory_helper import compute_module_sizes
module_size = compute_module_sizes(model)
print(f"The model size is {module_size[''] * 1e-9} GB")

The model size is 9.08120448 GB
